# CADE

In [174]:
import warnings
warnings.filterwarnings('ignore')

## Libraries import

In [175]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, auc, precision_recall_curve
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
import time

## Prediction translation

In [176]:
def def_outlier(df):
    if (df['y_pred'] in [-1]):
        val = 1
    else:
        val = 0
    return val

## CADE threshold

In [177]:
def def_outlier_cade(df, threshold):
    if (df['Target'] > threshold):
        val = 1
    else:
        val = 0
    return val

## Arrhythmia

**Dataset source**: http://odds.cs.stonybrook.edu/arrhythmia-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources**:

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

In [178]:
data = pd.read_csv('./arrhythmia.csv', sep = ',')

In [179]:
# dropping columns that consist only of 0's
data = data.drop(columns = ['Col15', 'Col63', 'Col65', 'Col79', 'Col127', 'Col128','Col135', 'Col137', 'Col139','Col141',
'Col147', 'Col152', 'Col153','Col160','Col200', 'Col260', 'Col270'])

In [180]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col265,Col266,Col267,Col268,Col269,Col271,Col272,Col273,Col274,y
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.9,2.9,23.3,49.4,1
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.3,3.4,12.3,49.0,0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.4,2.6,34.6,61.6,0
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,-0.1,3.9,25.4,62.8,1


In [181]:
data.shape

(452, 258)

In [182]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,386
1,66


### CADE

In [183]:
train_data = data.copy()

In [184]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )
end = time.process_time()
arrhythmia_cade_train_time = end - start
print(end - start)

start = time.process_time()
train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
arrhythmia_cade_test_time = end - start
print(end - start)

0.59375
0.0


In [185]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [186]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args = (max_key,), axis = 1)

In [187]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[350,  36],
       [ 46,  20]], dtype=int64)

In [188]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
arrhythmia_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6038035798398492

In [189]:
arrhythmia_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.88      0.91      0.90       386
           1       0.36      0.30      0.33        66

    accuracy                           0.82       452
   macro avg       0.62      0.60      0.61       452
weighted avg       0.81      0.82      0.81       452



In [190]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
arrhythmia_cade_auc_precision_recall = metrics.auc(recall, precision)
print(arrhythmia_cade_auc_precision_recall)

0.2930205978878545


## Cardiotocography

**Dataset source**: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [191]:
data = pd.read_csv('./Cardiotocography.csv')

In [192]:
data.shape

(1831, 22)

In [193]:
data['y'] = data['y'].astype(int)

In [194]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,1655
1,176


In [195]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,y
0,0.004912,0.693191,-0.203640,0.595322,0.353190,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0
1,0.110729,-0.079903,-0.203640,1.268942,0.396246,-0.061401,-0.278295,-1.710270,0.759072,-0.420487,...,-0.798376,1.854728,0.278625,0.963083,0.301464,0.129265,0.093563,-0.256385,-0.493294,0
2,0.216546,-0.272445,-0.203640,1.050988,0.148753,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,2.342663,-0.488279,0.061002,0.065417,0.024596,-0.256385,1.140018,0
3,0.004912,0.727346,-0.203640,1.212171,-0.683598,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,1.654650,-0.488279,0.061002,0.193113,0.093563,-0.323186,1.140018,0
4,-0.100905,0.363595,1.321366,1.027120,0.141359,-0.061401,-0.278295,-0.992364,-0.051613,-0.420487,...,-0.085638,-0.565334,0.278625,-0.488279,-0.059229,0.065417,0.024596,-0.456787,1.140018,0


### CADE

In [196]:
train_data = data.copy()

In [197]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )
end = time.process_time()
cardio_cade_train_time = end - start
print(end - start)

start = time.process_time()
train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
cardio_cade_test_time = end - start
print(end - start)

0.359375
0.03125


In [198]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [199]:
train_data['prediction'] = train_data.apply(def_outlier_cade, args = (max_key,), axis = 1)

In [200]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[1584,   71],
       [ 150,   26]], dtype=int64)

In [201]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
cardio_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.552942186212579

In [202]:
cardio_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1655
           1       0.27      0.15      0.19       176

    accuracy                           0.88      1831
   macro avg       0.59      0.55      0.56      1831
weighted avg       0.85      0.88      0.86      1831



In [203]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
cardio_cade_auc_precision_recall = metrics.auc(recall, precision)
print(cardio_cade_auc_precision_recall)

0.213020029525421


## ForestCover

**Dataset source**: http://odds.cs.stonybrook.edu/forestcovercovertype-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [204]:
data = pd.read_csv('./ForestCover.csv')

In [205]:
data.shape

(286048, 11)

In [206]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,283301
1,2747


In [207]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,y
0,2804,139,9,268,65,3180,234,238,135,6121,0
1,2785,155,18,242,118,3090,238,238,122,6211,0
2,2579,132,6,300,-15,67,230,237,140,6031,0
3,2886,151,11,371,26,5253,234,240,136,4051,0
4,2742,134,22,150,69,3215,248,224,92,6091,0


### CADE

In [208]:
train_data = data.copy()

In [209]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
forestcover_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
forestcover_cade_test_time = end - start
print(end - start)

175.484375
4.421875


In [210]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [211]:
train_data['prediction'] = train_data.apply(def_outlier_cade, args = (max_key,), axis = 1)

In [212]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[271138,  12163],
       [  2236,    511]], dtype=int64)

In [213]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
forestcover_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5720404933029851

In [214]:
forestcover_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97    283301
           1       0.04      0.19      0.07      2747

    accuracy                           0.95    286048
   macro avg       0.52      0.57      0.52    286048
weighted avg       0.98      0.95      0.97    286048



In [215]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
forestcover_cade_auc_precision_recall = metrics.auc(recall, precision)
print(forestcover_cade_auc_precision_recall)

0.029358242961903147


## Annthyroid

**Dataset source**: http://odds.cs.stonybrook.edu/annthyroid-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Aditional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [216]:
data = pd.read_csv('./annthyroid.csv')

In [217]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.73,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.00025,0.026,0.139,0.090,0.153,0


In [218]:
data.shape

(7200, 7)

In [219]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,6666
1,534


### CADE

In [220]:
train_data = data.copy()

In [221]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
annthyroid_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
annthyroid_cade_test_time = end - start
print(end - start)

1.1875
0.046875


In [222]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [223]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [224]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[6364,  302],
       [ 383,  151]], dtype=int64)

In [225]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
annthyroid_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6183931595406732

In [226]:
annthyroid_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      6666
           1       0.33      0.28      0.31       534

    accuracy                           0.90      7200
   macro avg       0.64      0.62      0.63      7200
weighted avg       0.90      0.90      0.90      7200



In [227]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
annthyroid_cade_auc_precision_recall = metrics.auc(recall, precision)
print(annthyroid_cade_auc_precision_recall)

0.23127011668519729


## Credit card 

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [228]:
data = pd.read_csv('./creditcard.csv')

In [229]:
data = data.drop(columns = ['Time'])

In [230]:
data.shape

(284807, 30)

In [231]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [232]:
pd.pivot_table(data,
             values = 'V1',
               index = 'Class', 
              aggfunc = 'count')

,V1
Class,
0,284315
1,492


### CADE

In [233]:
train_data = data.copy()

In [234]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'Class']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
creditcard_cade_train_time = end - start
print(end - start)

start = time.process_time()
train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'Class'])[:,1]
end = time.process_time()
creditcard_cade_test_time = end - start
print(end - start)

247.296875
2.09375


In [235]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['Class'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [236]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [237]:
confusion_matrix(train_data['Class'], train_data['prediction'])

array([[282683,   1632],
       [   321,    171]], dtype=int64)

In [238]:
fpr, tpr, _ = metrics.roc_curve(train_data['Class'], train_data['Target'])
creditcard_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6709123904852471

In [239]:
creditcard_cade_report = classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.09      0.35      0.15       492

    accuracy                           0.99    284807
   macro avg       0.55      0.67      0.57    284807
weighted avg       1.00      0.99      1.00    284807



In [240]:
precision, recall, thresholds = precision_recall_curve(train_data['Class'], train_data['Target'])
creditcard_cade_auc_precision_recall = metrics.auc(recall, precision)
print(creditcard_cade_auc_precision_recall)

0.062270609637990595


## Mammography

**Dataset source**: http://odds.cs.stonybrook.edu/mammography-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [241]:
data = pd.read_csv('./mammography.csv')

In [242]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [243]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,10923
1,260


In [244]:
data.shape

(11183, 7)

### CADE

In [245]:
train_data = data.copy()

In [246]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
mammography_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
mammography_cade_test_time = end - start
print(end - start)

1.765625
0.078125


In [247]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [248]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [249]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[10596,   327],
       [  181,    79]], dtype=int64)

In [250]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
mammography_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6386830188944993

In [251]:
mammography_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     10923
           1       0.19      0.30      0.24       260

    accuracy                           0.95     11183
   macro avg       0.59      0.64      0.61     11183
weighted avg       0.96      0.95      0.96     11183



In [252]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
mammography_cade_auc_precision_recall = metrics.auc(recall, precision)
print(mammography_cade_auc_precision_recall)

0.20337368706394565


## Shuttle

**Dataset source**: http://odds.cs.stonybrook.edu/shuttle-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [253]:
data = pd.read_csv('./shuttle.csv', sep = ',')

In [254]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,y
0,50,21,77,0,28,0,27,48,22,1
1,53,0,82,0,52,-5,29,30,2,0
2,37,0,76,0,28,18,40,48,8,0
3,37,0,79,0,34,-26,43,46,2,0
4,85,0,88,-4,6,1,3,83,80,1


In [255]:
data.shape

(49097, 10)

In [256]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,45586
1,3511


### CADE

In [257]:
train_data = data.copy()

In [258]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
shuttle_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
shuttle_cade_test_time = end - start
print(end - start)

12.484375
0.515625


In [259]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [260]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [261]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[45494,    92],
       [ 3409,   102]], dtype=int64)

In [262]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
shuttle_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5135101871545281

In [263]:
shuttle_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     45586
           1       0.53      0.03      0.06      3511

    accuracy                           0.93     49097
   macro avg       0.73      0.51      0.51     49097
weighted avg       0.90      0.93      0.90     49097



In [264]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
shuttle_cade_auc_precision_recall = metrics.auc(recall, precision)
print(shuttle_cade_auc_precision_recall)

0.3032456040328159


## mnist

**Dataset source**: http://odds.cs.stonybrook.edu/mnist-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Bandaragoda, Tharindu R., et al. “Efficient Anomaly Detection by Isolation Using Nearest Neighbour Ensemble.” 2014 IEEE International Conference on Data Mining Workshop. IEEE, 2014.

In [265]:
data = pd.read_csv('./mnist.csv')

In [266]:
data = data.drop(columns = ['Col1','Col4', 'Col7', 'Col22', 'Col27', 'Col29', 'Col38', 'Col41', 'Col51', 'Col53', 'Col54', 'Col61', 'Col62', 'Col71', 'Col73', 'Col79', 'Col87', 'Col88', 'Col89', 'Col90',
'Col92', 'Col100'])

In [267]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,6903
1,700


In [268]:
data.shape

(7603, 79)

In [269]:
data.head()

,Col2,Col3,Col5,Col6,Col8,Col9,Col10,Col11,Col12,Col13,...,Col86,Col91,Col93,Col94,Col95,Col96,Col97,Col98,Col99,y
0,-73.804153,198.205963,-13.124617,-1.1501,-0.141633,179.249390,114.661163,-80.736702,130.659348,162.649841,...,-15.392716,188.055649,-4.469967,158.381409,-137.100632,27.131416,-2.274633,-0.00065,-12.351267,0
1,-73.804153,197.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,128.659348,190.649841,...,-15.392716,186.055649,-4.469967,123.381416,-137.100632,157.131409,-2.274633,-0.00065,-12.351267,0
2,-73.804153,-53.794033,-13.124617,-1.1501,-0.141633,-73.750618,-44.338833,170.263306,130.659348,46.649849,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0
3,-73.804153,86.205963,-13.124617,-1.1501,-0.141633,76.249382,208.661163,107.263298,130.659348,190.649841,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,74.131416,-2.274633,-0.00065,-12.351267,0
4,-27.804153,199.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,130.659348,91.649849,...,-15.392716,188.055649,-4.469967,22.381416,-137.100632,159.131409,-2.274633,-0.00065,-12.351267,0


### CADE

In [270]:
train_data = data.copy()

In [271]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
mnist_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
mnist_cade_test_time = end - start
print(end - start)

1.296875
0.0625


In [272]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [273]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [274]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[6897,    6],
       [ 693,    7]], dtype=int64)

In [275]:
fpr, tpr, _ = metrics.roc_curve(data['y'], train_data['Target'])
mnist_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5045623021046749

In [276]:
mnist_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      6903
           1       0.54      0.01      0.02       700

    accuracy                           0.91      7603
   macro avg       0.72      0.50      0.49      7603
weighted avg       0.87      0.91      0.87      7603



In [277]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
mnist_cade_auc_precision_recall = metrics.auc(recall, precision)
print(mnist_cade_auc_precision_recall)

0.31531312646974224


## vowels

**Dataset source**: http://odds.cs.stonybrook.edu/japanese-vowels-data/

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [278]:
data = pd.read_csv('./vowels.csv')

In [279]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0.0,1406
1.0,50


In [280]:
data.shape

(1456, 13)

In [281]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,y
0,0.580469,-0.902534,0.617899,-0.997942,-2.463799,-0.846455,2.349849,0.375400,-0.649334,1.604637,-0.623060,-0.383125,0.0
1,0.784375,-1.077366,0.615781,-0.921911,-2.388553,-0.638047,2.106684,0.361018,-0.714317,1.260236,-0.423339,-0.287791,0.0
2,0.791292,-1.086242,0.669773,-0.806112,-2.260781,-0.538491,2.053282,0.266492,-0.842815,1.081797,-0.267201,-0.172203,0.0
3,1.217306,-1.083425,0.855483,-0.724879,-2.155552,-0.101879,1.768597,0.303151,-1.044710,0.655290,0.214298,-0.341840,0.0
4,1.065352,-1.030178,0.773297,-0.452289,-1.955907,0.248205,1.530474,0.253740,-0.968961,-0.208287,0.331578,0.007288,0.0


### CADE

In [282]:
train_data = data.copy()

In [283]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
vowels_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
vowels_cade_test_time = end - start
print(end - start)

0.90625
0.03125


In [284]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [285]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [286]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[1103,  303],
       [  12,   38]], dtype=int64)

In [287]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
vowels_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8345661450924609

In [288]:
vowels_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      0.78      0.88      1406
           1       0.11      0.76      0.19        50

    accuracy                           0.78      1456
   macro avg       0.55      0.77      0.53      1456
weighted avg       0.96      0.78      0.85      1456



In [289]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
vowels_cade_auc_precision_recall = metrics.auc(recall, precision)
print(vowels_cade_auc_precision_recall)

0.2652953960415727


## Seismic

**Dataset source**: http://odds.cs.stonybrook.edu/seismic-dataset/ (data is transformed from .arff to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [290]:
data = pd.read_csv('./seismic.csv', sep = ',')

In [291]:
data = data.drop(columns = ['nbumps6','nbumps7','nbumps89'])

In [292]:
data.shape

(2584, 16)

In [293]:
drop_enc = OneHotEncoder(drop='first').fit_transform(data[['seismic','seismoacoustic','shift','ghazard']])

In [294]:
cat_var = pd.DataFrame(drop_enc.toarray())
cat_var.columns = ['seismic: b', 'seismoacoustic: b','seismoacoustic: c','shift: W','ghazard: b','ghazard: c']

In [295]:
data = pd.concat([data, cat_var], axis = 1)

In [296]:
data = data.drop(columns = ['seismic','seismoacoustic','shift','ghazard'])

In [297]:
data.head()

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,maxenergy,class,seismic: b,seismoacoustic: b,seismoacoustic: c,shift: W,ghazard: b,ghazard: c
0,15180,48,-72,-72,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,14720,33,-70,-79,1,0,1,0,0,2000,2000,0,0.0,0.0,0.0,0.0,0.0,0.0
2,8050,30,-81,-78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,28820,171,-23,40,1,0,1,0,0,3000,3000,0,0.0,0.0,0.0,0.0,0.0,0.0
4,12640,57,-63,-52,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [298]:
pd.pivot_table(data,
             values = 'genergy',
               index = 'class', 
              aggfunc = 'count')

,genergy
class,
0,2414
1,170


### CADE

In [299]:
train_data = data.copy()

In [300]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'class']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
seismic_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'class'])[:,1]
end = time.process_time()
seismic_cade_test_time = end - start
print(end - start)

0.421875
0.015625


In [301]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [302]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [303]:
confusion_matrix(train_data['class'], train_data['prediction'])

array([[2394,   20],
       [ 168,    2]], dtype=int64)

In [304]:
fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['Target'])
seismic_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.501754471465471

In [305]:
seismic_cade_report = classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      2414
           1       0.09      0.01      0.02       170

    accuracy                           0.93      2584
   macro avg       0.51      0.50      0.49      2584
weighted avg       0.88      0.93      0.90      2584



In [306]:
precision, recall, thresholds = precision_recall_curve(train_data['class'], train_data['Target'])
seismic_cade_auc_precision_recall = metrics.auc(recall, precision)
print(seismic_cade_auc_precision_recall)

0.0791654939487757


## Musk

**Dataset source**: http://odds.cs.stonybrook.edu/musk-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

In [307]:
data = pd.read_csv('./musk.csv', sep = ',')

In [308]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col158,Col159,Col160,Col161,Col162,Col163,Col164,Col165,Col166,y
0,46.0,-108.0,-60.0,-69.0,-117.0,49.0,38.0,-161.0,-8.0,5.0,...,-308.0,52.0,-7.0,39.0,126.0,156.0,-50.0,-112.0,96.0,1.0
1,41.0,-188.0,-145.0,22.0,-117.0,-6.0,57.0,-171.0,-39.0,-100.0,...,-59.0,-2.0,52.0,103.0,136.0,169.0,-61.0,-136.0,79.0,1.0
2,46.0,-194.0,-145.0,28.0,-117.0,73.0,57.0,-168.0,-39.0,-22.0,...,-134.0,-154.0,57.0,143.0,142.0,165.0,-67.0,-145.0,39.0,1.0
3,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,136.0,168.0,-60.0,-135.0,80.0,1.0
4,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,137.0,168.0,-60.0,-135.0,80.0,1.0


In [309]:
data['y'] = data['y'].astype(int)

In [310]:
data.shape

(3062, 167)

In [311]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,2965
1,97


### CADE

In [312]:
train_data = data.copy()

In [313]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'y']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
musk_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'y'])[:,1]
end = time.process_time()
musk_cade_test_time = end - start
print(end - start)

2.40625
0.03125


In [314]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [315]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [316]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[2678,  287],
       [  69,   28]], dtype=int64)

In [317]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['Target'])
musk_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5997027172684758

In [318]:
musk_cade_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.90      0.94      2965
           1       0.09      0.29      0.14        97

    accuracy                           0.88      3062
   macro avg       0.53      0.60      0.54      3062
weighted avg       0.95      0.88      0.91      3062



In [319]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['Target'])
musk_cade_auc_precision_recall = metrics.auc(recall, precision)
print(musk_cade_auc_precision_recall)

0.09023421682044762


## bank

**Dataset source**: https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main/categorical%20data

Pang, G., Shen, C., Cao, L., & Hengel, A. V. D. (2021). Deep learning for anomaly detection: A review. ACM Computing Surveys (CSUR), 54(2), 1-38.

In [320]:
data = pd.read_csv('./bank.csv')

In [321]:
data.head()

,age,job=housemaid,job=services,job=admin.,job=blue-collar,job=technician,job=retired,job=management,job=unemployed,job=self-employed,...,previous,poutcome=nonexistent,poutcome=failure,poutcome=success,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,class
0,0.209877,0,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0,0,1,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,0.937500,0.698753,0.602510,0.957379,0.859735,0
3,0.160494,0,1,0,0,0,0,0,0,0,...,0.142857,0,1,0,0.333333,0.269680,0.192469,0.150759,0.512287,0
4,0.530864,0,0,0,1,0,0,0,0,0,...,0.000000,1,0,0,0.333333,0.340608,0.154812,0.174790,0.512287,1


In [322]:
pd.pivot_table(data,
             values = 'age',
               index = 'class', 
              aggfunc = 'count')

,age
class,
0,36548
1,4640


### CADE

In [323]:
train_data = data.copy()

In [324]:
start = time.process_time()

fake = pd.DataFrame()
data_for_fake = train_data.loc[:, train_data.columns != 'class']
data_for_fake['Target'] = 0
for i in data_for_fake.columns:
    fake[i] = np.random.uniform(np.min(data_for_fake[i]), np.max(data_for_fake[i]), size = len(data_for_fake[i]))
fake['Target'] = 1

data_combined = pd.concat([data_for_fake, fake], axis = 0)

model = RandomForestClassifier(n_estimators = 100)
model.fit(data_combined.loc[:, data_combined.columns != 'Target'],data_combined.loc[:, data_combined.columns == 'Target']['Target'] )

end = time.process_time()
bank_cade_train_time = end - start
print(end - start)

start = time.process_time()

train_data['Target'] = model.predict_proba(train_data.loc[:, train_data.columns != 'class'])[:,1]
end = time.process_time()
bank_cade_test_time = end - start
print(end - start)

6.125
0.1875


In [325]:
auc = {}
for i in np.arange(0,0.55,0.05):
    train_data['prediction'] = train_data.apply(def_outlier_cade, args=(i,), axis = 1)
    fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['prediction'])
    auc[i] = metrics.auc(fpr, tpr)
max_key = max(auc, key=auc.get)

In [326]:
train_data['prediction'] = train_data.apply(def_outlier_cade,args=(max_key,), axis = 1)

In [327]:
confusion_matrix(train_data['class'], train_data['prediction'])

array([[36540,     8],
       [ 4636,     4]], dtype=int64)

In [328]:
fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['Target'])
bank_cade_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5003216247504463

In [329]:
bank_cade_report = classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     36548
           1       0.33      0.00      0.00      4640

    accuracy                           0.89     41188
   macro avg       0.61      0.50      0.47     41188
weighted avg       0.82      0.89      0.83     41188



In [330]:
precision, recall, thresholds = precision_recall_curve(train_data['class'], train_data['Target'])
bank_cade_auc_precision_recall = metrics.auc(recall, precision)
print(bank_cade_auc_precision_recall)

0.22337622887594755


## Performance

In [331]:
performance = pd.DataFrame(columns = ['F1 score', 'recall', 'precision', 'AUC', 'AUPRC', 
                                      'Training time','Inference time','Total time'])

In [332]:
f1_score_cade = {'arrhythmia':arrhythmia_cade_report['1']['f1-score'],
                       'cardio':cardio_cade_report['1']['f1-score'], 
                        'forestcover':forestcover_cade_report['1']['f1-score'], 
                       'annthyroid':annthyroid_cade_report['1']['f1-score'],       
                        'creditcard':creditcard_cade_report['1']['f1-score'], 
                       'mammography':mammography_cade_report['1']['f1-score'], 
                        'shuttle':shuttle_cade_report['1']['f1-score'], 
                      'mnist':mnist_cade_report['1']['f1-score'], 
                  'vowels':vowels_cade_report['1']['f1-score'], 
                  'seismic':seismic_cade_report['1']['f1-score'], 
                  'musk':musk_cade_report['1']['f1-score'], 
                  'bank':bank_cade_report['1']['f1-score']}
f1_score_cade_df = pd.DataFrame.from_dict(f1_score_cade, orient='index', columns = ['F1 score']).reset_index()

In [333]:
recall_cade = {'arrhythmia':arrhythmia_cade_report['1']['recall'],
                       'cardio':cardio_cade_report['1']['recall'], 
                        'forestcover':forestcover_cade_report['1']['recall'], 
                       'annthyroid':annthyroid_cade_report['1']['recall'],       
                        'creditcard':creditcard_cade_report['1']['recall'], 
                       'mammography':mammography_cade_report['1']['recall'], 
                        'shuttle':shuttle_cade_report['1']['recall'], 
                      'mnist':mnist_cade_report['1']['recall'], 
                  'vowels':vowels_cade_report['1']['recall'], 
                  'seismic':seismic_cade_report['1']['recall'], 
                  'musk':musk_cade_report['1']['recall'], 
                  'bank':bank_cade_report['1']['recall'], }
recall_cade_df = pd.DataFrame.from_dict(recall_cade, orient='index', columns = ['Recall']).reset_index()

In [334]:
precision_cade = {'arrhythmia':arrhythmia_cade_report['1']['precision'],
                       'cardio':cardio_cade_report['1']['precision'], 
                        'forestcover':forestcover_cade_report['1']['precision'], 
                       'annthyroid':annthyroid_cade_report['1']['precision'],       
                        'creditcard':creditcard_cade_report['1']['precision'], 
                       'mammography':mammography_cade_report['1']['precision'], 
                        'shuttle':shuttle_cade_report['1']['precision'], 
                      'mnist':mnist_cade_report['1']['precision'], 
                  'vowels':vowels_cade_report['1']['precision'], 
                  'seismic':seismic_cade_report['1']['precision'], 
                  'musk':musk_cade_report['1']['precision'], 
                  'bank':bank_cade_report['1']['precision'], }
precision_cade_df = pd.DataFrame.from_dict(precision_cade, orient='index', columns = ['Precision']).reset_index()

In [335]:
auc_cade = {'arrhythmia':arrhythmia_cade_auc,
                       'cardio':cardio_cade_auc, 
                        'forestcover':forestcover_cade_auc, 
                       'annthyroid':annthyroid_cade_auc,       
                        'creditcard':creditcard_cade_auc, 
                       'mammography':mammography_cade_auc, 
                        'shuttle':shuttle_cade_auc, 
                      'mnist':mnist_cade_auc, 
                  'vowels':vowels_cade_auc, 
                  'seismic':seismic_cade_auc, 
                  'musk':musk_cade_auc, 
                  'bank':bank_cade_auc}
auc_cade_df = pd.DataFrame.from_dict(auc_cade, orient='index', columns = ['AUC']).reset_index()

In [336]:
auprc_cade = {'arrhythmia':arrhythmia_cade_auc_precision_recall,
                       'cardio':cardio_cade_auc_precision_recall, 
                        'forestcover':forestcover_cade_auc_precision_recall, 
                       'annthyroid':annthyroid_cade_auc_precision_recall,       
                        'creditcard':creditcard_cade_auc_precision_recall, 
                       'mammography':mammography_cade_auc_precision_recall, 
                        'shuttle':shuttle_cade_auc_precision_recall, 
                      'mnist':mnist_cade_auc_precision_recall, 
                  'vowels':vowels_cade_auc_precision_recall, 
                  'seismic':seismic_cade_auc_precision_recall, 
                  'musk':musk_cade_auc_precision_recall, 
                  'bank':bank_cade_auc_precision_recall}
auprc_cade_df = pd.DataFrame.from_dict(auprc_cade, orient='index', columns = ['AUPRC']).reset_index()

In [337]:
training_time_cade = {'arrhythmia':arrhythmia_cade_train_time,
                       'cardio':cardio_cade_train_time, 
                        'forestcover':forestcover_cade_train_time, 
                       'annthyroid':annthyroid_cade_train_time,       
                        'creditcard': creditcard_cade_train_time, 
                       'mammography':mammography_cade_train_time, 
                        'shuttle':shuttle_cade_train_time, 
                      'mnist':mnist_cade_train_time, 
                  'vowels':vowels_cade_train_time, 
                  'seismic':seismic_cade_train_time, 
                  'musk':musk_cade_train_time, 
                  'bank':bank_cade_train_time}
training_time_cade_df = pd.DataFrame.from_dict(training_time_cade, orient='index', columns = ['Training time']).reset_index()

In [338]:
test_time_cade = {'arrhythmia':arrhythmia_cade_test_time,
                       'cardio':cardio_cade_test_time, 
                        'forestcover':forestcover_cade_test_time, 
                       'annthyroid':annthyroid_cade_test_time,       
                        'creditcard': creditcard_cade_test_time, 
                       'mammography':mammography_cade_test_time, 
                        'shuttle':shuttle_cade_test_time, 
                      'mnist':mnist_cade_test_time, 
                  'vowels':vowels_cade_test_time, 
                  'seismic':seismic_cade_test_time, 
                  'musk':musk_cade_test_time, 
                  'bank':bank_cade_test_time}
test_time_cade_df = pd.DataFrame.from_dict(test_time_cade, orient='index', columns = ['Testing time']).reset_index()

In [339]:
total_time_cade = {'arrhythmia':arrhythmia_cade_train_time + arrhythmia_cade_test_time,
                       'cardio':cardio_cade_train_time + cardio_cade_test_time, 
                        'forestcover':forestcover_cade_train_time + forestcover_cade_test_time, 
                       'annthyroid':annthyroid_cade_train_time + annthyroid_cade_test_time,       
                        'creditcard': creditcard_cade_train_time + creditcard_cade_test_time, 
                       'mammography':mammography_cade_train_time + mammography_cade_test_time, 
                        'shuttle':shuttle_cade_train_time + shuttle_cade_test_time, 
                      'mnist':mnist_cade_train_time + mnist_cade_test_time, 
                  'vowels':vowels_cade_train_time + vowels_cade_test_time, 
                  'seismic':seismic_cade_train_time + seismic_cade_test_time, 
                  'musk':musk_cade_train_time + musk_cade_test_time, 
                  'bank':bank_cade_train_time + bank_cade_test_time}
total_time_cade_df = pd.DataFrame.from_dict(total_time_cade, orient='index', columns = ['Total time']).reset_index()

In [340]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_cade_df, recall_cade_df, how = 'inner'), 
                                    precision_cade_df, how ='inner'),
         auc_cade_df, how = 'inner'), auprc_cade_df, how = 'inner'), training_time_cade_df, how = 'inner'), 
         test_time_cade_df, how = 'inner'),total_time_cade_df, how = 'inner')

,index,F1 score,Recall,Precision,AUC,AUPRC,Training time,Testing time,Total time
0,arrhythmia,0.327869,0.303030,0.357143,0.603804,0.293021,0.593750,0.000000,0.593750
1,cardio,0.190476,0.147727,0.268041,0.552942,0.213020,0.359375,0.031250,0.390625
2,forestcover,0.066273,0.186021,0.040319,0.572040,0.029358,175.484375,4.421875,179.906250
3,annthyroid,0.305978,0.282772,0.333333,0.618393,0.231270,1.187500,0.046875,1.234375
4,creditcard,0.149020,0.347561,0.094842,0.670912,0.062271,247.296875,2.093750,249.390625
5,mammography,0.237237,0.303846,0.194581,0.638683,0.203374,1.765625,0.078125,1.843750
6,shuttle,0.055061,0.029052,0.525773,0.513510,0.303246,12.484375,0.515625,13.000000
7,mnist,0.019635,0.010000,0.538462,0.504562,0.315313,1.296875,0.062500,1.359375
8,vowels,0.194373,0.760000,0.111437,0.834566,0.265295,0.906250,0.031250,0.937500
9,seismic,0.020833,0.011765,0.090909,0.501754,0.079165,0.421875,0.015625,0.437500
